In [1]:
import sys, os, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, os.path.join(parent_dir,'spring-break'))
sys.path.insert(0, os.path.join(parent_dir,'Linear Classifier'))

In [2]:
import numpy as np
import random
import copy
import math
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from martins.complex_transformer import ComplexTransformer
from FNNLinear import FNNLinear
from FNNSeparated import FNNSeparated
from GAN import Generator, Discriminator
from data_utils import *
import argparse
import logging
import logging.handlers
import pickle
from centerloss import CenterLoss
from DataSetLoader import JoinDataset, SingleDataset
from torch.autograd import Variable
from binaryloss import BinaryLoss

# Parser

In [3]:
# Parameters
parser = argparse.ArgumentParser(description='JDA Time series adaptation')
parser.add_argument("--data_path", type=str, default="../data_unzip", help="dataset path")
parser.add_argument("--task", type=str, help='3A or 3E')
parser.add_argument('--gpu_num', type=int, default=0, help='gpu number')
parser.add_argument('--batch_size', type=int, default=256, help='batch size')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--seed', type=int, default=0, help='manual seed')
parser.add_argument('--lr_centerloss', type=float, default=0.005, help='learning rate for centerloss')
parser.add_argument('--target_lbl_percentage', type=float, default=0.7, help='percentage of which target data has label')
parser.add_argument('--source_lbl_percentage', type=float, default=0.7, help='percentage of which source data has label')
parser.add_argument('--num_per_class', type=int, default=-1, help='number of sample per class when training local discriminator')
parser.add_argument('--save_path', type=str, default='../train_related/', help='where to store data')
parser.add_argument('--gfunction_epoch', type=int, default=5000, help='epoch of which gfunction is trained for')
parser.add_argument('--KL', type=bool, default=False, help="if calculate KL divergence")
parser.add_argument('--JS', type=bool, default=False, help="if calculate JS divergence")


args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH] [--task TASK]
                             [--gpu_num GPU_NUM] [--batch_size BATCH_SIZE]
                             [--epochs EPOCHS] [--lr_FNN LR_FNN]
                             [--lr_encoder LR_ENCODER]
                             [--lr_centerloss LR_CENTERLOSS] [--sclass SCLASS]
                             [--scent SCENT] [--lr_gan LR_GAN]
                             [--target_lbl_percentage TARGET_LBL_PERCENTAGE]
                             [--source_lbl_percentage SOURCE_LBL_PERCENTAGE]
                             [--num_per_class NUM_PER_CLASS] [--seed SEED]
                             [--save_path SAVE_PATH]
                             [--model_save_period MODEL_SAVE_PERIOD]
                             [--epoch_begin_prototype EPOCH_BEGIN_PROTOTYPE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Tianqin Li\AppData\Roaming\jupyter\runtime\kernel-460988ff-287e-4235-b5eb-83de570d2508.json


SystemExit: 2

C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')

# seed
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
cudnn.deterministic = True
torch.backends.cudnn.deterministic = True


args.task = '3Av2' if args.task == '3A' else '3E'
num_class = 50 if args.task == "3Av2" else 65
device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')

if args.num_per_class == -1:
    args.num_per_class = math.ceil(args.batch_size / num_class)
    
model_sub_folder = '/f-gan/%s_domain'%args.task
if args.KL: model_sub_folder += '_KL'
if args.JS: model_sub_folder += '_JS'   

if not os.path.exists(args.save_path+model_sub_folder):
    os.makedirs(args.save_path+model_sub_folder)

# Logger

In [8]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

if os.path.isfile(args.save_path+model_sub_folder+ '/logfile.log'):
    os.remove(args.save_path+model_sub_folder+ '/logfile.log')
    
file_log_handler = logging.FileHandler(args.save_path+model_sub_folder+ '/logfile.log')
logger.addHandler(file_log_handler)

stdout_log_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stdout_log_handler)

attrs = vars(args)
for item in attrs.items():
    logger.info("%s: %s"%item)


data_path: ../data_unzip
task: 3E
num_class: 50
batch_size: 100
num_per_class: 2
gap: 5
lbl_percentage: 0.7
lr_gan: 0.0001
lr_FNN: 0.0001
lr_encoder: 0.0001
epochs: 2
clip_value: 0.01
n_critic: 4
sclass: 0.7
scent: 0.01
seed: 0
save_path: ../train_related
model_save_period: 1
lr_centerloss: 0.001
lr_prototype: 0.001
sprototype: 0.01
select_pretrain_epoch: 77
epoch_begin_prototype: 0
sbinary_loss: 1
gpu_num: 0
source_lbl_percentage: 0.7
target_lbl_percentage: 0.7


# Data Loading


In [ ]:
data_dict = np.load(args.data_path+'processed_file_not_one_hot_%s.pkl'%args.task, allow_pickle=True)

source_x = data_dict['tr_data']
target_x = data_dict['te_data']

# join_dataset = JoinDataset(labeled_x, labeled_y, unlabeled_x, unlabeled_y, random=True)
# join_dataloader = DataLoader(join_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4)


# Weight initialize

In [10]:
def weights_init(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
    elif type(m) == nn.LayerNorm:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)


# Model creation

In [ ]:
class Gfunction(nn.Sequential):
    def __init__(self):
        super(Gfunction, self).__init__(
            nn.Linear(3200,1600),
            nn.ELU(),
            nn.Linear(1600,800),
            nn.ELU(),
            nn.Linear(800,400),
            nn.ELU(),
            nn.Linear(400,200),
            nn.ELU(),
            nn.Linear(200,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,100),
            nn.ELU(),
            nn.Linear(100,1),
        )

In [ ]:
def log_mean_exp(x, device):
    max_score = x.max()
    batch_size = torch.Tensor([x.shape[0]]).to(device)
    stable_x = x - max_score
    return max_score - batch_size.log() + stable_x.exp().sum(dim=0).log()

a = torch.rand([100,1]).to(device)
assert torch.all(log_mean_exp(a, device) - a.exp().mean(dim=0).log() < 1e-6)

In [276]:
def KLDiv(g_x_source, g_x_target, device):
    # clipping
#     g_x_source = torch.clamp(g_x_source, -1e3, 1e3)
#     g_x_target = torch.clamp(g_x_target, -1e3, 1e3)
    return g_x_source.mean(dim=0) - log_mean_exp(g_x_target, device)

In [ ]:
def JSDiv(g_x_source, g_x_target, device):
    return -F.softplus(-g_x_source).mean(dim=0) - F.softplus(g_x_target).mean(dim=0)

In [ ]:
device = torch.device('cuda:{}'.format(args.gpu_num) if torch.cuda.is_available() else 'cpu')
print(device)

if args.KL:
    gfunction_KL_div = Gfunction().to(device)
    gfunction_KL_div.apply(weights_init)
    optimizer_gfunction_KL_div = torch.optim.Adam(gfunction_KL_div.parameters(), lr=args.lr)

if args.JS:
    gfunction_JS_div = Gfunction().to(device)
    gfunction_JS_div.apply(weights_init)
    optimizer_gfunction_JS_div = torch.optim.Adam(gfunction_JS_div.parameters(), lr=args.lr)



# Train

In [16]:
source_x = torch.Tensor(source_x.reshape(-1, 3200)).float().to(device)
target_x = torch.Tensor(target_x.reshape(-1, 3200)).float().to(device)

KL_div = []
JS_div = []
for i in tqdm(range(args.gfunction_epoch)):
#     for j, (labeled_x, labeled_y, unlabeled_x, unlabeled_y) in enumerate(join_dataloader):
#     labeled_x = labeled_x.to(device).float().view(-1, 3200)
#     unlabeled_x = unlabeled_x.to(device).float().view(-1, 3200)
    if args.KL:
        optimizer_gfunction_KL_div.zero_grad()
        x_source_g = gfunction_KL_div(source_x)
        x_target_g = gfunction_KL_div(target_x)
        loss_KL = - KLDiv(x_source_g, x_target_g, device) # maximize
        loss_KL.backward()
        optimizer_gfunction_KL_div.step()
        loss_KL = -loss_KL.item()
        KL_div.append(loss_KL)

    if args.JS:
        optimizer_gfunction_JS_div.zero_grad()
        x_source_g = gfunction_KL_div(source_x)
        x_target_g = gfunction_KL_div(target_x)
        loss_JS = - JSDiv(x_source_g, x_target_g, device) # maximize
        loss_JS.backward()
        optimizer_gfunction_JS_div.step()
        loss_JS = -loss_JS.item()
        JS_div.append(loss_JS)
            
    if i % 200 == 0:
        log_string = "Epoch %i: "%i
        if args.KL: log_string += "KL: %f; "%(loss_KL)
        if args.JS: log_string += "JS: %f; "%(loss_JS)   
        logger.info(log_string)
    
    if args.KL: 
        np.save(args.save_path+model_sub_folder+'/KL_div.npy', KL_div)
        
    if args.JS:
        np.save(args.save_path+model_sub_folder+'/JS_div.npy', JS_div)
        
log_string = "Epoch %i: "%i
if args.KL: log_string += "KL: %f; "%(loss_KL_labeled)
if args.JS: log_string += "JS: %f; "%(loss_JS_labeled)   
logger.info(log_string)
        


Started Training


  0%|          | 0/99 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 31, in _pin_memory_loop
    data = pin_memory(data)
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 55, in pin_memory
    return [pin_memory(sample) for sample in data]
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 55, in <listcomp>
    return [pin_memory(sample) for sample in data]
  File "C:\Users\Tianqin Li\anaconda3\envs\russ-local\lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 47, in pin_memory
    return data.pin_memory()
RuntimeError: cuda runtime error (710) : device-side assert triggered at ..\aten\src\THC\THCCachingHostAllocator.cpp:278
